In [16]:
import scipy
from scipy import stats
from scipy.stats import ttest_ind
from scipy.stats import t
import numpy as np
import plotly.graph_objects as go
import pandas as pd

# Задание 1

Реализовать формулу подсчета длительности теста, сравнить ее с онлайн калькуляторами (например https://mindbox.ru/tools/ab-test-calculator/ ). При сравнении оценить мощность критерия при указанном изменении и рассчитанном количестве наблюдений в выборке. 

In [4]:
def duration(k, delta_effect, sigma_1, sigma_2, alpha=0.05, beta=0.2):
    # k - group ratio
    # delta_effect -  expected change from mean i.e. effect size
    # sigma_1 - standard deviation of first group
    # sigma_2 standart deviation of second group
    # alpha - first type error
        # beta - second type error
    z = stats.norm.ppf(1 - alpha/2) + stats.norm.ppf(1-beta)
    n = (k+1) * z ** 2 * (sigma_1 ** 2 + sigma_2 **2 / k) / (delta_effect ** 2)
    return n

Параметры онлайн калькулятора:
- Количество вариантов - количество групп (=2)
- Средний показатель
- Ожидаемый абсолютный прирост =  delta_effect
- Достоверность = 1 - alpha / 2 * 100%
- Мощность =   1 - beta * 100%

In [12]:
duration(k=1, delta_effect=0.1, sigma_1=0.1, sigma_2=0.2)

78.48879734349089

In [6]:
print(f"pow with beta is {(1 - 0.2) *100}%")

pow with beta is 80.0%


Для калькулятора с аналогичными параметрами и средним = 20 % результат показал : 396 человек всего, 198 в каждом варианте

In [11]:
IT_NUM = 78
ALPHA = 0.05
def check_p_val(gen1, stat_crit, effect = 0, scale_eff = 0, loc = 0, scale = 1,  size = 10000):
    p_vals = []
    for i in range(IT_NUM):
        arr_1 = gen1(loc = loc, scale = scale, size = size)
        arr_2 = gen1(loc = loc + effect , scale = scale + scale_eff, size = size)
        res = stat_crit(arr_1, arr_2, alternative='two-sided')
        p_vals.append(res.pvalue)
    p_vals = np.array(p_vals)
    pow = p_vals[p_vals < ALPHA].shape[0] / p_vals.shape[0] * 100
    print(f"criteria power is {pow}")
    
gen1 = stats.norm.rvs
crit =  stats.ttest_ind
check_p_val(gen1, crit, effect = 0.1, scale_eff=0.1, loc = 0.2, scale = 0.1, size = 198)

criteria power is 100.0


На практике мощность получилась даже больше при заданных показателях

# Задание 2

Реализовать метод линеаризации. Проверить для него корректность и мощность. Мощность должна быть больше, чем просто на обычных значениях конверсии пользователей.

In [20]:
IT_NUM = 1000
ARR_SIZE = 100
ALPHA = 0.05

In [41]:
p_val_lin = []
p_val = []
for i in range(IT_NUM):
    data = []
    views = stats.expon.rvs(loc = 100, scale = 100, size = ARR_SIZE * 2)
    for view in views:
        if len(data) < ARR_SIZE:
            clicks = np.sum(stats.bernoulli.rvs(p=0.05, size=int(view)))
            data.append((int(view), clicks, clicks / int(view), "A"))
        else:
            clicks = np.sum(stats.bernoulli.rvs(p=0.055, size=int(view)))
            data.append((int(view), clicks, clicks / int(view), "B"))
    df = pd.DataFrame(data, columns = ["views", "clicks", "cr", "group"])
    cr_a = df.loc[df["group"] == "A", "clicks"].sum() / df.loc[df["group"] == "A", "views"].sum()
    df["lin_ratio"] = df["clicks"] - cr_a *  df["views"]
    a_lin = df.loc[df["group"] == "A", "lin_ratio"]
    b_lin = df.loc[df["group"] == "B", "lin_ratio"]
    p_value_lin = stats.ttest_ind(a_lin, b_lin).pvalue
    p_value = stats.ttest_ind(df.loc[df["group"] == "A", "cr"], df.loc[df["group"] == "B", "cr"]).pvalue
    p_val_lin.append(p_value_lin)
    p_val.append(p_value)

In [42]:
p_val = np.array(p_val)
p_val_lin = np.array(p_val_lin)
pow = p_val[p_val < ALPHA].shape[0] / p_val.shape[0] * 100
pow_lin = p_val_lin[p_val_lin < ALPHA].shape[0] / p_val_lin.shape[0] * 100
print(f"pow value is {pow}")
print(f"pow value for linear ratio is {pow_lin}")

pow value is 51.5
pow value for linear ratio is 60.6


Линейный критерий мощнее

In [60]:
p_val_lin = []
p_val = []
for i in range(IT_NUM):
    data = []
    views = stats.expon.rvs(loc = 100, scale = 100, size = ARR_SIZE * 2)
    for view in views:
        if len(data) < ARR_SIZE:
            clicks = np.sum(stats.bernoulli.rvs(p=0.05, size=int(view)))
            data.append((int(view), clicks, clicks / int(view), "A"))
        else:
            clicks = np.sum(stats.bernoulli.rvs(p=0.05, size=int(view)))
            data.append((int(view), clicks, clicks / int(view), "B"))
    df = pd.DataFrame(data, columns = ["views", "clicks", "cr", "group"])
    cr_a = df.loc[df["group"] == "A", "clicks"].sum() / df.loc[df["group"] == "A", "views"].sum()
    df["lin_ratio"] = df["clicks"] - cr_a *  df["views"]
    a_lin = df.loc[df["group"] == "A", "lin_ratio"]
    b_lin = df.loc[df["group"] == "B", "lin_ratio"]
    p_value_lin = stats.ttest_ind(a_lin, b_lin).pvalue
    p_value = stats.ttest_ind(df.loc[df["group"] == "A", "cr"], df.loc[df["group"] == "B", "cr"]).pvalue
    p_val_lin.append(p_value_lin)
    p_val.append(p_value)
p_val = np.array(p_val)
p_val_lin = np.array(p_val_lin)

График корректности обычной конверсии

In [61]:
probs = []
x = [0.01 * i for i in range(101)]
for i in range(101):
    alpha_step = 0.01 * i
    probs.append(p_val[p_val < alpha_step].shape[0] / p_val.shape[0])

fig = go.Figure([go.Scatter(x=x, y=probs, mode="markers", name = "p_value"),
                 go.Scatter(x=x,y=x, mode="lines",name="uniform")])
fig.update_layout(height=600, width=600, title="Q-Q plot")

График корректности с использованием линеаризации

In [62]:
probs = []
x = [0.01 * i for i in range(101)]
for i in range(101):
    alpha_step = 0.01 * i
    probs.append(p_val_lin[p_val_lin < alpha_step].shape[0] / p_val_lin.shape[0])

fig = go.Figure([go.Scatter(x=x, y=probs, mode="markers", name = "p_value"),
                 go.Scatter(x=x,y=x, mode="lines",name="uniform")])
fig.update_layout(height=600, width=600, title="Q-Q plot")

Оба графика похожи, линеаризация визуально более сбалансирована вокруг прямой

# Задание 3

Реализовать метод CUPED. Проверить для него корректность и мощность. Данные на этапе до A/B тесте необходимо сгенерировать один раз, далее синтетически генерировать только часть, связанную с проведением A/B-теста.


In [53]:
p_val_cup = []
p_val = []
pre_data = stats.norm.rvs(loc = 100, scale = 20, size = ARR_SIZE)
for i in range(IT_NUM):
    arr_1 = stats.norm.rvs(loc = 1, scale = 0.1, size = ARR_SIZE ) *  pre_data
    arr_2 = stats.norm.rvs(loc = 1 + 0.1, scale = 0.1, size = ARR_SIZE ) *  pre_data
    theta = np.cov(pre_data, arr_1)[0, 1] / np.std(pre_data)**2 
    arr_1_cup = arr_1 - theta  * pre_data
    arr_2_cup = arr_2 - theta * pre_data
    p_value_cup = stats.ttest_ind(arr_1_cup, arr_2_cup).pvalue
    p_value = stats.ttest_ind(arr_1, arr_2).pvalue
    p_val_cup.append(p_value_cup)
    p_val.append(p_value) 

In [54]:
p_val = np.array(p_val)
p_val_cup = np.array(p_val_cup)
pow = p_val[p_val < ALPHA].shape[0] / p_val.shape[0] * 100
pow_cup = p_val_cup[p_val_cup < ALPHA].shape[0] / p_val_cup.shape[0] * 100
print(f"pow value is {pow}")
print(f"pow value for CUPED is {pow_cup}")

pow value is 98.8
pow value for CUPED is 100.0


CUPED мощнее

График корректности обычной конверсии

In [68]:
p_val_cup = []
p_val = []
pre_data = stats.norm.rvs(loc = 100, scale = 20, size = ARR_SIZE)
for i in range(IT_NUM):
    arr_1 = stats.norm.rvs(loc = 1, scale = 0.1, size = ARR_SIZE ) *  pre_data
    arr_2 = stats.norm.rvs(loc = 1, scale = 0.1, size = ARR_SIZE ) *  pre_data
    theta = np.cov(pre_data, arr_1)[0, 1] / np.std(pre_data)**2 
    arr_1_cup = arr_1 - theta  * pre_data
    arr_2_cup = arr_2 - theta * pre_data
    p_value_cup = stats.ttest_ind(arr_1_cup, arr_2_cup).pvalue
    p_value = stats.ttest_ind(arr_1, arr_2).pvalue
    p_val_cup.append(p_value_cup)
    p_val.append(p_value) 
p_val = np.array(p_val)
p_val_cup = np.array(p_val_cup)

In [69]:
probs = []
x = [0.01 * i for i in range(101)]
for i in range(101):
    alpha_step = 0.01 * i
    probs.append(p_val[p_val < alpha_step].shape[0] / p_val.shape[0])

fig = go.Figure([go.Scatter(x=x, y=probs, mode="markers", name = "p_value"),
                 go.Scatter(x=x,y=x, mode="lines",name="uniform")])
fig.update_layout(height=600, width=600, title="Q-Q plot")

График конверсии CUPED

In [70]:
probs = []
x = [0.01 * i for i in range(101)]
for i in range(101):
    alpha_step = 0.01 * i
    probs.append(p_val_cup[p_val_cup < alpha_step].shape[0] / p_val_cup.shape[0])

fig = go.Figure([go.Scatter(x=x, y=probs, mode="markers", name = "p_value"),
                 go.Scatter(x=x,y=x, mode="lines",name="uniform")])
fig.update_layout(height=600, width=600, title="Q-Q plot")

CUPED предзакания показали куда более лучший результат , если данные скореллированы